# EXTRACT

In [5]:
dir_query = os.path.join(path, 'sql', 'all_boards_ATIVOS.sql')
with open(dir_query, 'r') as file:
    query = file.read()
df_ativacoes = awr.athena.read_sql_query(query, database='silver')





In [6]:
df_ativacoes.shape[0]

114685

In [11]:
df_ativacoes_ativos = df_ativacoes[df_ativacoes['status_beneficio']=='ATIVO']
df_ativacoes_ativos_semdup = df_ativacoes_ativos.drop_duplicates(subset=['chassi'])
df_ativacoes_ativos_semdup = df_ativacoes_ativos_semdup[df_ativacoes_ativos_semdup['status_beneficio']=='ATIVO']
df_ativacoes_ativos_semdup.shape[0]

33548

In [12]:
# TRANSFORM

# DEFININDO DATAFRAMES VAZIOS 
df_final_ativacoes_concat = pd.DataFrame()


# Assegure que df_cancelamentos e df_ativacoes já existem no escopo global antes de rodar este bloco!
# TRANSFORMANDO DF_ATIV E SEGMENTANDO POR EMPRESA


df_ativ_all_boards = df_ativacoes

df_ativ_all_boards['data_ativacao_beneficio'] = pd.to_datetime(df_ativ_all_boards['data_ativacao_beneficio']).dt.date

df_ativ_all_boards['beneficio'] = (
    df_ativ_all_boards['beneficio']
    .replace('REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO', 'CASCO (VEÍCULO)')
    .replace('REPARAÇÃO OU REPOSIÇÃO DO (SEMI)REBOQUE', 'CASCO (R/SR)')
    .replace('REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO', 'CASCO (COMPLEMENTO)')
)

df_ativ_viavante = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Viavante']
df_ativ_stcoop = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Stcoop']
df_ativ_segtruck = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Segtruck']
df_ativ_tag = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Tag']

# SELECIONANDO APENAS AS ATIVAÇÕES CORRESPONDENTES AOS BENEFICIOS 'CASCO' / 'TERCEIRO' POR UM REGEX PADRÃO
ids_beneficios_segtruck = [2, 3, 4, 7, 24, 25, 26, 29]
ids_beneficios_stcoop = [24, 25, 26, 29]
ids_beneficios_viavante = [40, 41, 42, 45]
ids_beneficios_tag = [2, 3, 4, 7, 24, 25, 26, 29, 34, 35, 36, 37, 38, 39]

df_ativ_viavante = df_ativ_viavante.loc[df_ativ_viavante['id_beneficio'].isin(ids_beneficios_viavante)]
df_ativ_stcoop = df_ativ_stcoop[df_ativ_stcoop['id_beneficio'].isin(ids_beneficios_stcoop)]
df_ativ_segtruck = df_ativ_segtruck.loc[df_ativ_segtruck['id_beneficio'].isin(ids_beneficios_segtruck)]
df_ativ_tag = df_ativ_tag.loc[df_ativ_tag['id_beneficio'].isin(ids_beneficios_tag)]

# CONCATENANDO E CRIANDO COLUNA DE MIGRAÇÃO (MIGRATION_FROM) 
df_final_ativacoes_concat = pd.concat([df_ativ_viavante, df_ativ_stcoop, df_ativ_segtruck, df_ativ_tag])


In [ ]:

if not df_final_ativacoes_concat.empty:
    df_final_ativacoes_concat['migration_from'] = np.nan

# DEFININDO COLUNAS QUE SERÃO UTILIZADAS NO DATAFRAME FINAL
df_final_ativacoes_concat = df_final_ativacoes_concat[[
    'placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria', 'matricula', 'conjunto', 'unidade', 'consultor', 'status_beneficio', 
    'cliente', 'data_registro', 'data_ativacao_beneficio', 'suporte', 'data_filtro', 'empresa', 'migration_from'
]]

# RETIRANDO DUPLICATAS
df_final_ativacoes_concat = df_final_ativacoes_concat.drop_duplicates(subset=['chassi'])

# TRATANDO DADOS NULOS NOS DATAFRAMES
df_final_ativacoes_concat['placa'] = df_final_ativacoes_concat['placa'].fillna('SEM-PLACA')
df_final_ativacoes_concat['chassi'] = df_final_ativacoes_concat['chassi'].fillna('NULL')
df_final_ativacoes_concat['id_placa'] = df_final_ativacoes_concat['id_placa'].fillna(0)
df_final_ativacoes_concat['id_veiculo'] = df_final_ativacoes_concat['id_veiculo'].fillna(0)
df_final_ativacoes_concat['id_carroceria'] = df_final_ativacoes_concat['id_carroceria'].fillna(0)
df_final_ativacoes_concat['matricula'] = df_final_ativacoes_concat['matricula'].fillna(0)
df_final_ativacoes_concat['conjunto'] = df_final_ativacoes_concat['conjunto'].fillna(0)
df_final_ativacoes_concat['unidade'] = df_final_ativacoes_concat['unidade'].fillna('NULL')
df_final_ativacoes_concat['consultor'] = df_final_ativacoes_concat['consultor'].fillna('NULL')
df_final_ativacoes_concat['status_beneficio'] = df_final_ativacoes_concat['status_beneficio'].fillna('NULL')
df_final_ativacoes_concat['cliente'] = df_final_ativacoes_concat['cliente'].fillna('NULL')
df_final_ativacoes_concat['data_registro'] = df_final_ativacoes_concat['data_registro'].fillna(pd.Timestamp('1900-01-01'))
df_final_ativacoes_concat['data_ativacao_beneficio'] = df_final_ativacoes_concat['data_ativacao_beneficio'].fillna(pd.Timestamp('1900-01-01'))
df_final_ativacoes_concat['suporte'] = df_final_ativacoes_concat['suporte'].fillna('NULL')
df_final_ativacoes_concat['data_filtro'] = df_final_ativacoes_concat['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
df_final_ativacoes_concat['empresa'] = df_final_ativacoes_concat['empresa'].fillna('NULL')
df_final_ativacoes_concat['migration_from'] = df_final_ativacoes_concat['migration_from'].fillna('NULL')

# LOAD

# Assegure que today já existe no escopo global antes de rodar este bloco!
file_path = rf"C:\Users\raphael.almeida\Documents\Processos\placas_acompanhamento\template\placas_movimentacoes.xlsx"

destination_dir = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas"
destination_path = os.path.join(destination_dir, os.path.basename(file_path))

destination_dir2 = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\bkp_activation"
destination_path2 = os.path.join(destination_dir2, os.path.basename(file_path))

os.makedirs(destination_dir, exist_ok=True)
os.makedirs(destination_dir2, exist_ok=True)

with pd.ExcelWriter(destination_path, engine='openpyxl') as writer:
    df_final_ativacoes_concat.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')


with pd.ExcelWriter(destination_path2, engine='openpyxl') as writer:
    df_final_ativacoes_concat.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')




In [16]:
df_final_ativacoes_concat_semdup = df_final_ativacoes_concat.drop_duplicates(subset=['chassi'])
df_final_ativacoes_concat_semdup.shape[0]

33025